In [0]:
# Upload File
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
# Read data content
f =open("/content/wdbc.data", "r")
if f.mode == 'r':
    dataContents =f.readlines()

dataContents

In [0]:
# Creating list of list of strings for data content
splitedContents = [i.split(",") for i in dataContents]
splitedContents

In [0]:
# Creating dataFrame
import pandas as pd

dataFrame = pd.DataFrame(splitedContents)
dataFrame

In [0]:
# Create Histogram plot for class 'M' and class 'B'
import matplotlib.pyplot as plt

#tmp = dataframe for class B only
tmp = dataFrame.loc[dataFrame[1] == 'B',:]

#tmp2 = dataframe for class M only
tmp2 = dataFrame.loc[dataFrame[1] == 'M',:]

plt.hist(tmp)
plt.show()

In [0]:
# Create Histogram plot for column class only
import matplotlib.pyplot as plt
# extract column of the classes from dataframe
Class = dataFrame.loc[:,1]
Class

plt.hist(Class)
plt.show()

In [0]:
# remove first and second columns (instance id and class) 
data = dataFrame.drop(columns=0)
data = data.drop(columns=1)
data

In [0]:
# create scatter plot for the first attribute
import numpy

plt.figure(figsize=(52,2))
plt.scatter(dataFrame.loc[1:100,2],dataFrame.loc[1:100,1])
plt.show()

In [0]:
#create attribute list to create box plot
attributesList = []
i = 2
j = 0
while i < 32:
    listTemp = data.loc[:,i].values.tolist()
    listTemp2 = [float(k) for k in listTemp]
    attributesList.insert(j , listTemp2)
    i = i + 1
    j = j + 1

attributesList

In [0]:
# Create boxplot
plt.figure(figsize=(20,25))
plt.boxplot(attributesList)
plt.show()

In [0]:
#Calculate correlation coefficient for Attribute List
import numpy
correlationMatrix = numpy.corrcoef(attributesList)
correlationMatrix

In [0]:
#Visualize correlationMatrix using imshow
plt.imshow(correlationMatrix)

In [0]:
#split data set to training data (70% of data, 398 instances) and test data (30% of data, 171 instances)
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=4, test_size=0.3,train_size=0.7, random_state=1)

#convert data and Class (dataframes) to arrays
dataArray = numpy.array(data)
classArray = numpy.array(Class)

#sss.get_n_splits(dataArray, classArray)

for train_index, test_index in sss.split(dataArray, classArray):
#    print("TRAIN:", train_index, "TEST:", test_index)
    data_train, data_test = dataArray[train_index], dataArray[test_index]
    Class_train, Class_test = classArray[train_index], classArray[test_index]

print("-----------------------------")
print(len(data_train))
print("==========")
print(data_train)
print("-----------------------------")
print(len(data_test))
print("==========")
print(data_test)
print("-----------------------------")
print(len(Class_train))
print("==========")
print(Class_train)
print("-----------------------------")
print(len(Class_test))
print("==========")
print(Class_test)

In [0]:
# create train dataframe
trainDataFrame = pd.DataFrame(data_train)
trainDataFrame

In [0]:
# create train data attribute list
trainAttributeList = []
i = 0
while i < 30:
    listTemp = trainDataFrame.loc[:,i].values.tolist()
    listTemp2 = [float(k) for k in listTemp]
    trainAttributeList.insert(i , listTemp2)
    i = i + 1

trainAttributeList

In [0]:
# data train minmax normalization
from sklearn.preprocessing import minmax_scale

normalizedMinmaxTrain = minmax_scale(trainAttributeList)

print(normalizedMinmaxTrain)

plt.boxplot(trainAttributeList[0])

plt.show()

normalizedMinmaxTrainAttribute = minmax_scale(trainAttributeList[0])

plt.boxplot(normalizedMinmaxTrainAttribute)

plt.show()


In [0]:
# create test dataframe
testDataFrame = pd.DataFrame(data_test)
testDataFrame

In [0]:
# create test data attribute list
testAttributeList = []
i = 0
while i < 30:
    listTemp = testDataFrame.loc[:,i].values.tolist()
    listTemp2 = [float(k) for k in listTemp]
    testAttributeList.insert(i , listTemp2)
    i = i + 1

testAttributeList

In [0]:
# data test minmax normalization
from sklearn.preprocessing import minmax_scale

normalizedMinmaxTest = minmax_scale(testAttributeList)

print(normalizedMinmaxTest)

plt.boxplot(testAttributeList[0])

plt.show()

normalizedMinmaxTestAttribute = minmax_scale(testAttributeList[0])

plt.boxplot(normalizedMinmaxTestAttribute)

plt.show()

In [0]:
# create DecisionTreeClassifier model
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

DTC_clf = DecisionTreeClassifier(max_depth=5)

DTC_clf = DTC_clf.fit(data_train,Class_train)

DTC_Class_pred = DTC_clf.predict(data_test)

print("Accuracy:",metrics.accuracy_score(Class_test, DTC_Class_pred))
print(metrics.confusion_matrix(Class_test, DTC_Class_pred))
print(metrics.classification_report(Class_test, DTC_Class_pred,target_names=['M','B']))

In [0]:
# create AdaBoostClassifier model
from sklearn.ensemble import AdaBoostClassifier
# Create adaboost classifer object
ABC_clf = AdaBoostClassifier(n_estimators=50,learning_rate=1)
# Train Adaboost Classifer
model = ABC_clf.fit(data_train, Class_train)

#Predict the response for test dataset
ABC_Class_pred = model.predict(data_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Class_test, ABC_Class_pred))
print(metrics.confusion_matrix(Class_test, ABC_Class_pred))
print(metrics.classification_report(Class_test, ABC_Class_pred,target_names=['M','B']))

In [0]:
# create RandomForestClassifier model
from sklearn.ensemble import RandomForestClassifier

RFC_clf=RandomForestClassifier(n_estimators=100,max_depth=5)

RFC_clf.fit(data_train, Class_train)

RFC_Class_pred = RFC_clf.predict(data_test)

print("Accuracy:",metrics.accuracy_score(Class_test, RFC_Class_pred))
print(metrics.confusion_matrix(Class_test, RFC_Class_pred))
print(metrics.classification_report(Class_test, RFC_Class_pred,target_names=['M','B']))

In [0]:
#apply GridSearchCV on each classfifier model to get best parameters
from sklearn.model_selection import GridSearchCV

DTC_params = { 
        'max_depth' : [4,5,6,7,8,9,10]
}

ABC_params = { 
    'n_estimators': [10, 50, 80, 100]
}

RFC_params = { 
    'n_estimators': [10, 50, 80, 100],
    'max_depth' : [4,5,6,7,8,9,10]
}

CV_DTC = GridSearchCV(estimator=DTC_clf, param_grid=DTC_params, cv= 10)
CV_DTC.fit(data_train, Class_train)
print("DTC_best_parameters",CV_DTC.best_params_ )

CV_ABC = GridSearchCV(estimator=ABC_clf, param_grid=ABC_params, cv= 10)
CV_ABC.fit(data_train, Class_train)
print("ABC_best_parameters",CV_ABC.best_params_ )

CV_RFC = GridSearchCV(estimator=RFC_clf, param_grid=RFC_params, cv= 10)
CV_RFC.fit(data_train, Class_train)
print("RFC_best_parameters",CV_RFC.best_params_ )